In [ ]:
import numpy as np
import os
import random
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, LSTM, RepeatVector, Dropout, TimeDistributed

class LSTM_Model():
    def __init__(self, X=None, Y=None, num_units=200, epochs=10, batch_size=64,output_dim=None , name=""):
        self.X = X
        self.Y = Y
        self.num_units = num_units
        self.epochs = epochs
        self.batch_size = batch_size
        self.name = name
        self.timesteps = 16 # one bar
        self.output_dim = output_dim
    def create_model(self):
        print("create model...")
        
        model = Sequential()
        model.add(LSTM(self.num_units, input_shape=(self.timesteps, self.X.shape[-1]),
                       return_sequences=True))
        model.add(Dropout(0.2))
        model.add(LSTM(self.num_units, return_sequences=True))
        model.add(Dropout(0.2))
        model.add(Dense(self.Y.shape[2], activation='softmax'))
        
        model.compile(loss='categorical_crossentropy', optimizer='adam')
        model.summary()
        model.save('models/' + self.name + '.h5')
    def model2(self):
        print("Another model...")
        
        input_notes = Input(shape=(self.timesteps, self.X.shape[-1]))
        rs = True
        lstm = LSTM(200, return_sequences=rs, name="lstm_" + str(1))(input_notes)
        lstm = (Dropout(0.2))(lstm)

        predict = Dense(200, activation='relu')(lstm)
    #     output = Dense(len(note_type_arr), activation='softmax')(predict)
        output = Dense(self.Y.shape[2], activation='softmax')(predict)

        model = Model(input_notes, output)
        model.compile(optimizer='adam', loss='categorical_crossentropy')


        model.summary()
        model.save('models/' + self.name + '.h5')
        #model.fit_generator(chorale_dict, steps_per_epoch=32, epochs=10)
        #model.fit(X,X, batch_size=2, epochs=10)
    def train_model(self, gen, test_gen):
        print("train model...")
        model = load_model('models/' + self.name + '.h5')
        model.compile(loss='categorical_crossentropy', optimizer='adam')
        model.fit_generator(gen, steps_per_epoch=256, epochs=self.epochs, validation_data=(test_gen), validation_steps=16)
        model.save('models/' + self.name + '.h5')
    def generator(self, notes, onehot_notes, part_i):
        t = 0
        while True:
            X = []
            Y = []
            if t < notes.shape[0] - 16:
                X.append(notes[t:t+16])
                Y.append(onehot_notes[t:t+16,part_i,:])
            else: 
                t = 0
                continue
            t += 16
            X = np.array(X)
            Y = np.array(Y)
            yield (X,Y)
